<h1 align='center'>Laboratorio de Fuga</h1>

<h2>1. Importación de Librerías</h2>

Para simplificar el ejercicio, primero importaremos las librerías, sin necesidad de levantar un ambiente virtual e instalarlas en éste, gracias a las virtudes de Google Colaboratory como editor. Otra de las ventajas de  este intérprete de Python, es que funciona con el formato de celdas de los *Jupyter Notebooks*. Esto permite trabajar bajo el paradigma de lo que se denomina *Literate Programming*, pudiéndose hacer un claro énfasis en la estructura lógica del programa.

Para ejecutar la celda a continuación, bastará que usted la seleccione apretando sobre ella con el cursor, y luego apriete <code>shift+enter</code>




In [1]:
# Statistical Libraries
import numpy as np
import statsmodels.api as sm

# Operational Libraries
import pandas as pd
from typing import Optional

<h2>2. Lectura de la Tabla de Datos</h2>

Para ejercitar, utilizaremos una tabla de datos (en adelante <i>dataset</i>) proporcionada por la empresa de Telecom, perteneciente a la industria de las telecomunicaciones en Chile. Los datos no han sido procesados, por lo que hay ciertos campos que se transformarán y otros tantos que enriqueceremos en el proceso de segmentación.

Es importante notar que hay una columna numérica de identificación de los sujetos, denominada "Identificador". No se confunda, esta variable es arbitraria para todos los efectos prácticos, y no debe ser considerada para el análisis, salvo que sea considerada como llave relacional. Si usted no se encuentra familiarizado con el lenguaje y las librerías, Pandas incluye siempre un índice que comúnmente es también numérico. No confunda el índice con la columna de identificación.

In [4]:
#Importing the prospects dataset using pandas
file_path = f'propuesto_tymo.xlsx'
dataset_tymo = pd.read_excel(file_path, sheet_name='propuesto_tymo')

# Display of last 5 rows
dataset_tymo.head(5)

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.1,0


Vamos a analizar las distintas columnas de las que disponemos, utilizando funciones como las que se muestran a continuación:

In [5]:
for col in dataset_tymo.columns:
    unique_values = dataset_tymo[col].unique()

    if len(unique_values) < 5:
        print(f'{col}: {unique_values}')

    else:
        print(f'{col}: {unique_values[:5]} (muestra)')

CustomerId: [15634602 15647311 15619304 15701354 15737888] (muestra)
Surname: ['Hargrave' 'Hill' 'Onio' 'Boni' 'Mitchell'] (muestra)
CreditScore: [619 608 502 699 850] (muestra)
Geography: ['France' 'Spain' 'Germany']
Gender: ['Female' 'Male']
Age: [42 41 39 43 44] (muestra)
Tenure: [2 1 8 7 4] (muestra)
Balance: [     0.    83807.86 159660.8  125510.82 113755.78] (muestra)
NumOfProducts: [1 3 2 4]
HasCrCard: [1 0]
IsActiveMember: [1 0]
EstimatedSalary: ['101348.88' '112542.58' '113931.57' '93826.63' '79084.1'] (muestra)
Exited: [1 0]


Gracias al análisis anterior, podemos tener una visión más clara respecto al contenido de cada columna, pudiéndose construir una tabla que las describra, como se muestra a continuación (Pendiente: crear nueva tabla):

| Columna | Nombre de la Variable | Contenido de la Columna | Medida o Alternativas |
|---------|----------------------|-------------------------|-----------------------|
| 1       | customerID           | Identificador único del cliente | 15634602, 15647311, 15619304, 15701354, ... |
| 2       | Surname              | Apellido del cliente | Hargrave, Hill, Onio, Boni, ... |
| 3       | CreditScore          | Puntaje de crédito del cliente | 619, 608, 502, 699, 850 ... |
| 4       | Geography            | País de residencia del cliente | France, Spain, Germany |
| 5       | gender               | Género del cliente | Female, Male |
| 6       | Age                  | Edad del cliente | 42, 41, 39, 43, 44, ... |
| 7       | Tenure               | Número de meses que el cliente ha estado en el servicio | 2, 1, 8, 7, 4, ... |
| 8       | Balance              | Saldo de la cuenta del cliente | 0.00, 83807.86, 159660.80, 125510.82, 113755.78, ... |
| 9       | NumOfProducts        | Número de productos que el cliente tiene contratados | 1, 3, 2, 4 |
| 10      | HasCrCard            | Indica si el cliente tiene tarjeta de crédito | 1, 0 |
| 11      | IsActiveMember       | Indica si el cliente es un miembro activo | 1, 0 |
| 12      | EstimatedSalary      | Salario estimado del cliente | 101348.88, 112542.58, 113931.57, 93826.63, 79084.1, ... |
| 13      | Exited               | Indica si el cliente ha abandonado el servicio | 1, 0 |



<h2>3. Transformación de los Datos</h2>

A continuación procederemos a transformar los datos, creando un dataset numérico a partir del que descargamos. Si bien es más costoso en memoria el uso de réplicas completas de los datasets utilizados, esta práctica es conveniente cuando se trabaja con Jupyter Notebooks que podrían ser ejecutados en desorden o múltiples veces.

In [6]:
# Copying our dataset to avoid future issues
dataset_tymo_numerico = dataset_tymo.copy(deep=True)

Recordemos que los datos recolectados en una encuesta pueden ser de cuatro tipos principalmente:

- Nominal: nombres (identificación y clasificación)
- Ordinal: orden (jerarquización, posición relativa)
- Intervalo: cuantificación (cero arbitrario)
- Escala: cuantificación (cero absoluto)

Es importante que usted identifique el tipo de cada variable, para que así le sea más sencillo transformar los datos en información valiosa para su posterior análisis. Para efectos de este ejemplo, dividiremos la transformación de los datos por tipo para mayor claridad, transformando aquellas variables de tipo texto en numéricas según corresponda.

In [8]:
cols_nominal = ['customerID', 'gender', 'SeniorCitizen', 'Partner',
                'Dependents', 'PhoneService', 'MultipleLines',
                'InternetService', 'OnlineSecurity', 'OnlineBackup',
                'DeviceProtection', 'TechSupport', 'StreamingTV',
                'StreamingMovies', 'Contract', 'PaperlessBilling',
                'PaymentMethod', 'Churn']

cols_nominal = ['customerID', 'Surname', 'Geography', 'gender', 'HasCrCard', 'IsActiveMember', 'Exited']

cols_ordinal = ['Tenure', 'NumOfProducts']

cols_escala = [ 'CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']


cols_escala = [ 'tenure', 'MonthlyCharges', 'TotalCharges']

<h3>3.1. Valores Nominales</h3>

Convenientemente, Pandas puede transformar cualquier variable de tipo nominal en una variable categórica por nosotros. Sin embargo, este proceso no está excento de errores y debe ser monitoreado de cerca. Para controlar la calidad de los datos, almacenaremos las categorías en una lista con un diccionario que nos permitirá mapear cada uno de los valores.

In [9]:
print(f'Identificamos {len(cols_nominal)} variables nominales de forma manual\n'
      'A continuación procedereos a transformarlas a variables numéricas\n')
dict_mappers = dict()
counter = 1

for col in cols_nominal:
    # Create and store our mapper dictionary, and print it for analysis
    mapping = dict(enumerate(dataset[col].astype('category').cat.categories))
    dict_mappers.update({col: {value: key for key, value in mapping.items()}})
    counter += 1

    # Print the mapper dictionary for max 5 entries of the dictionary
    mapper_print = {entrance for i, entrance
                    in enumerate(dict_mappers[col]) if i < 6}

    print(f'{counter}) {col}: {mapper_print}')

Identificamos 18 variables nominales de forma manual
A continuación procedereos a transformarlas a variables numéricas

2) customerID: {'0013-EXCHZ', '0004-TLHLJ', '0003-MKNFE', '0013-MHZWF', '0002-ORFBO', '0011-IGKFF'}
3) gender: {'Male', 'Female'}
4) SeniorCitizen: {0, 1}
5) Partner: {'No', 'Yes'}
6) Dependents: {'No', 'Yes'}
7) PhoneService: {'No', 'Yes'}
8) MultipleLines: {'No', 'Yes', 'No phone service'}
9) InternetService: {'DSL', 'Fiber optic', 'No'}
10) OnlineSecurity: {'No', 'No internet service', 'Yes'}
11) OnlineBackup: {'No', 'No internet service', 'Yes'}
12) DeviceProtection: {'No', 'No internet service', 'Yes'}
13) TechSupport: {'No', 'No internet service', 'Yes'}
14) StreamingTV: {'No', 'No internet service', 'Yes'}
15) StreamingMovies: {'No', 'No internet service', 'Yes'}
16) Contract: {'Month-to-month', 'One year', 'Two year'}
17) PaperlessBilling: {'No', 'Yes'}
18) PaymentMethod: {'Credit card (automatic)', 'Bank transfer (automatic)', 'Electronic check', 'Mailed chec

Un ejemplo de variables que podríamos querer codificar de otra forma serían MultiLines, OnlineSecurity, OnlineBackup, DeviceProtection, TechSupport, StreamingTV, y StreamingMovies. Esto se debe a que, independiente la razón por la que cumpla o no con tener los productos a los que hace referencia cada una de las variables, podemos codificar de forma booleana. Procederemos a ajustar estas variables a continuación:

In [10]:
forcing_booleans = ['MultipleLines', 'OnlineSecurity', 'OnlineBackup',
                    'DeviceProtection', 'TechSupport', 'StreamingTV',
                    'StreamingMovies']

for col in forcing_booleans:
    dict_mappers[col] = {value: 1 if value == 'Yes' else 0
                         for value in dict_mappers[col]}

    print(f"{col}: {dict_mappers[col]}")

MultipleLines: {'No': 0, 'No phone service': 0, 'Yes': 1}
OnlineSecurity: {'No': 0, 'No internet service': 0, 'Yes': 1}
OnlineBackup: {'No': 0, 'No internet service': 0, 'Yes': 1}
DeviceProtection: {'No': 0, 'No internet service': 0, 'Yes': 1}
TechSupport: {'No': 0, 'No internet service': 0, 'Yes': 1}
StreamingTV: {'No': 0, 'No internet service': 0, 'Yes': 1}
StreamingMovies: {'No': 0, 'No internet service': 0, 'Yes': 1}


Ahora podemos reemplazar los valores de cada columna por su equivalente numérico pasando por el mapper adecuado.

In [11]:
for col, mapper in dict_mappers.items():
    dataset_numerico[col] = dataset[col].map(mapper)

dataset_numerico.tail(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
7038,4853,1,0,1,1,24,1,1,0,1,...,1,1,1,1,1,1,3,"84,8","1990,5",0
7039,1525,0,0,1,1,72,1,1,1,0,...,1,0,1,1,1,1,1,"103,2","7362,9",0
7040,3367,0,0,1,1,11,0,0,0,1,...,0,0,0,0,0,1,2,"29,6","346,45",0
7041,5934,1,1,1,0,4,1,1,1,0,...,0,0,0,0,0,1,3,"74,4","306,6",1
7042,2226,1,0,0,0,66,1,0,1,1,...,1,1,1,1,2,1,0,"105,65","6844,5",0


<h3>3.2. Variables Ordinales</h3>

No identificamos variables ordinales para este ejercicio, aunque se podría argumentar que el tipo de contrato se podría ordenar de menor a mayor duración, cuestión que en nuestro caso pasó por defecto. En tal caso, la única diferencia es que tendríamos que aplicar los mappers en forma manual, forzando así nuestro criterio subjetivo sobre los datos.

<h3>3.3. Variables de Intervalo</h3>

Comúnmente las encuestas contendrán preguntas de tipo Likert con alternativas que buscan evaluar la opinión o satisfacción de los clientes respecto a ciertos tópicos. En este caso, no se dispone de este tipo de preguntas, cuestión que de cara a determinar la fuga de clientes podría ser perjudicial, pues contar con variables psicográficas o conductuales de los clientes no es sólo importante para la segmentación, sino también para este tipo de ejercicios.

<h3>3.4. Variables de Escala</h3>

También de libro, las variables de escala suelen ser la edad y montos finitos como en este caso el costo de los planes. Convenientemente, como todas estas variables ya son numéricas y contínuas, no debiéramos realizar ningún tipo de transformación. Sin embargo, como observaremos a continuación, las columnas asociadas son de tipo "object", cuestión que hace referencia a que los valores se almacenaron como texto y no como número.

In [12]:
mapper_escala = {'tenure': int, 'MonthlyCharges': float, 'TotalCharges': float}

for col, dtype in mapper_escala.items():
    if dtype == float:
        dataset_numerico[col] = dataset[col].str.replace(',', '.').astype(dtype)
    else:
        dataset_numerico[col] = dataset[col].astype(dtype)

    print(f"{col}: {dataset_numerico[col].dtype}")

tenure: int32
MonthlyCharges: float64
TotalCharges: float64


<h2>4. Ejecución de las Regresiones Logísticas</h2>

La regresión logística es un modelo estadístico vital para predecir la probabilidad de fuga de clientes, un fenómeno también conocido como churn, que afecta significativamente los ingresos y la rentabilidad de las empresas. Este modelo predice una variable categórica binaria a partir de variables independientes que describen características y comportamientos de los clientes, permitiendo calcular la probabilidad de fuga. Al aplicar regresiones logísticas, es posible identificar las variables con mayor impacto en la fuga, lo que ayuda a las empresas a comprender y mitigar los factores que la propician. Además, este modelo facilita la generación de una puntuación de probabilidad de fuga por cliente, optimizando las estrategias de retención al priorizar y personalizar las intervenciones. En definitiva, la regresión logística ofrece una herramienta esencial para la toma de decisiones informadas y la implementación de acciones efectivas para retener a los clientes en riesgo.

<h3>4.1. Prueba inicial y Correcciones</h3>

La idea de aquí en adelante es ir reduciendo las variables significativas para el análisis, mediante un proceso de ensayo y error en el que se van eliminando aquellas variables que presenten p-values no significativos. Sin embargo, siempre es bueno probar inicialmente a ver si nuestras columnas se acomodan al modelo estadístico.

In [13]:
dataset_regression = dataset_numerico.copy(deep=True)

try:
    # Run a logistic regression model with all variables and the churn as the target
    x = dataset_regression.drop(columns=['customerID', 'Churn'])
    y = dataset_regression['Churn']

    x = sm.add_constant(x)
    model = sm.Logit(y, x)
    result = model.fit()

    print(result.summary())

except Exception as e:
    print(e)

exog contains inf or nans


En este caso, observamos que las variables exógenas presentan valores infinitos o nulos. Revisaremos la estadísticas descriptiva de la tabla para revisar los máximos y descartar al primer sospechoso de que no podamos correr el modelo.

In [14]:
dataset_regression.describe()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,...,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7032.000000,7043.000000
mean,3521.000000,0.504756,0.162147,0.483033,0.299588,32.371149,0.903166,0.421837,0.872923,0.286668,...,0.343888,0.290217,0.384353,0.387903,0.690473,0.592219,1.574329,64.761692,2283.300441,0.265370
std,2033.283305,0.500013,0.368612,0.499748,0.458110,24.559481,0.295752,0.493888,0.737796,0.452237,...,0.475038,0.453895,0.486477,0.487307,0.833755,0.491457,1.068104,30.090047,2266.771362,0.441561
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.250000,18.800000,0.000000
25%,1760.500000,0.000000,0.000000,0.000000,0.000000,9.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,35.500000,401.450000,0.000000
50%,3521.000000,1.000000,0.000000,0.000000,0.000000,29.000000,1.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2.000000,70.350000,1397.475000,0.000000
75%,5281.500000,1.000000,0.000000,1.000000,1.000000,55.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,89.850000,3794.737500,1.000000
max,7042.000000,1.000000,1.000000,1.000000,1.000000,72.000000,1.000000,1.000000,2.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,3.000000,118.750000,8684.800000,1.000000


Dado que no hay valores infinitos, seguramente tengamos valores nulos en alguna columna. Este análisis es un poco más complejo, pero nada que no se pueda resolver consultando a Chat GPT si no supiéramos cómo hacerlo. A continuación, revisamos cada columna y buscamos la presencia de valores nulos.

In [15]:
for col in dataset_regression.columns:
    if dataset_regression[col].isna().sum() > 0:
        print(f'{col}: {dataset_regression[col].isna().sum()}')

display(dataset_regression[dataset_regression['TotalCharges'].isna()][['tenure', 'MonthlyCharges', 'TotalCharges']])

TotalCharges: 11


,tenure,MonthlyCharges,TotalCharges
488,0,52.55,NaN
753,0,20.25,NaN
936,0,80.85,NaN
1082,0,25.75,NaN
1340,0,56.05,NaN
3331,0,19.85,NaN
3826,0,25.35,NaN
4380,0,20.00,NaN
5218,0,19.70,NaN
6670,0,73.35,NaN


Observamos que los valores faltantes en la columna TotalCharges corresponden a clientes que no han pagado aún, por lo que podemos reemplazar estos valores por 0 o asumir que el cliente no ha pagado aún y reemplazarlos por la columna MonthlyCharges. Si aún le quedan dudas, realice la siguiente operación: `dataset_regression[dataset_regression['tenure']==0]`

In [16]:
dataset_regression['TotalCharges'] = dataset_regression['TotalCharges'].fillna(0)

Hint: "puede" que en el laboratorio que usted deba resolver no sean ni los valores infinitos ni los valores nulos el problema. "Quizás" haya variables fuertemente correlacionadas que "probablemente" usted podría identificar corriendo un correlograma. Se "sugiere" ese método para botar las variables problemáticas, aunque encontraría genial si usted identifica la verdadera causa del problema.

<h3>4.2. Ejecución de la regresión principal</h3>

Para las siguientes líneas, por favor asegúrese de instalar matplotlib y jinja2

In [17]:
dataset_optimized = dataset_regression.copy(deep=True)
dataset_optimized.drop(columns=['customerID'], inplace=True)
confidence_level = 0.05
counter = 1

def optimize_regression(dataset, confidence_level: Optional[float] = 0.05):
    # Drop the constant column
    if 'const' in dataset.columns: # if it exists
        dataset.drop(columns=['const'], inplace=True)

    # Run a logistic regression model with all variables and the churn as the target
    x = dataset.drop(columns=['Churn'])
    y = dataset['Churn']

    x = sm.add_constant(x)
    model = sm.Logit(y, x)
    result = model.fit(disp=False)
    drop_cols = {}

    # Access the p-values of the model
    for index in result.pvalues.index:
        if result.pvalues[index] >= confidence_level:
            drop_cols[index] = result.pvalues[index]

    return (result, {col: p_value for col, p_value in drop_cols.items() if col != 'const'})

while True:
    print(f'Iteración {counter}:\n')
    result, drop_cols = optimize_regression(dataset_optimized, confidence_level)
    print(result.summary())

    if len(drop_cols) == 0:
        print('\nEl modelo ha sido exitosamente optimizado\n')
        break
    else:
        counter += 1

    # Drop the column with the highest p-value
    column_to_drop = max(drop_cols, key=drop_cols.get)
    dataset_optimized.drop(columns=column_to_drop, inplace=True)
    drop_cols.pop(column_to_drop)

    print(f'\nSe eliminó la columna {column_to_drop}\n\n')

Iteración 1:

                           Logit Regression Results                           
Dep. Variable:                  Churn   No. Observations:                 7043
Model:                          Logit   Df Residuals:                     7023
Method:                           MLE   Df Model:                           19
Date:                Tue, 21 May 2024   Pseudo R-squ.:                  0.2812
Time:                        17:40:09   Log-Likelihood:                -2929.0
converged:                       True   LL-Null:                       -4075.1
Covariance Type:            nonrobust   LLR p-value:                     0.000
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.6864      0.183     -3.752      0.000      -1.045      -0.328
gender              -0.0274      0.065     -0.424      0.672      -0.154       0.099
SeniorCitizen 

<h2>Evaluar la efectividad del Modelo</h2>



Estas líneas de código evalúan nuestro modelo de predicción de fuga. La funcion evaluate_model convierte probabilidades en valores binarios, calcula la matriz de confusión. La matriz de confusión es clave, ya que muestra la cantidad de verdaderos positivos, verdaderos negativos, falsos positivos y falsos negativos, permitiendo identificar donde el modelo acierta y falla, proporcionando información valiosa para evaluar financieramente el costo asociado a la fuga.

In [18]:
# Definir la función de evaluación
def evaluate_model(model, x, y):
    # Obtener los valores predichos
    y_pred_prob = model.predict(sm.add_constant(x))

    # Convertir las probabilidades a valores binarios
    y_pred = [1 if value > 0.5 else 0 for value in y_pred_prob]

    # Calcular la matriz de confusión
    confusion_matrix = pd.crosstab(y, y_pred, rownames=['Actual'], colnames=['Predicted'])

    # Calcular la exactitud
    accuracy = (confusion_matrix[0][0] + confusion_matrix[1][1]) / len(y)

    return accuracy, confusion_matrix

# Preparar los datos (usando el mismo conjunto de datos que el modelo entrenado)
x = dataset_optimized.drop(columns=['Churn'])
y = dataset_optimized['Churn']

# Evaluar el modelo
accuracy, confusion_matrix = evaluate_model(result, x, y)
print(f'La exactitud del modelo es de {accuracy * 100:.2f}%\n')
display(confusion_matrix)

La exactitud del modelo es de 80.29%



Predicted,0,1
Actual,,
0,4634,540
1,848,1021


<h2>Efectos Porcentuales</h2>

Por último, transformaremos los coeficientes en odds ratios y en porcentajes, para poder interpretar de mejor manera el efecto de cada factor sobre la fuga de clientes.

In [19]:
# Obtener los coeficientes
coefficients = result.params

# Calcular los odds ratios
odds_ratios = np.exp(coefficients)

# Calcular los porcentajes de cambio
percentages = (odds_ratios - 1) * 100

# Mostrar los resultados
results_df = pd.DataFrame({
    'Coefficient': coefficients,
    'Odds Ratio': odds_ratios,
    'Percentage Change': percentages
})

print(results_df)

                  Coefficient  Odds Ratio  Percentage Change
const               -0.613755    0.541315         -45.868541
SeniorCitizen        0.250066    1.284110          28.410953
Dependents          -0.165214    0.847712         -15.228768
tenure              -0.058676    0.943012          -5.698751
PhoneService        -1.220204    0.295170         -70.483015
OnlineSecurity      -0.573195    0.563722         -43.627826
OnlineBackup        -0.320304    0.725928         -27.407193
DeviceProtection    -0.233620    0.791662         -20.833764
TechSupport         -0.576967    0.561599         -43.840089
Contract            -0.739572    0.477318         -52.268164
PaperlessBilling     0.369623    1.447188          44.718835
MonthlyCharges       0.031153    1.031643           3.164348
TotalCharges         0.000300    1.000300           0.029982


<h2>Respuesta a la pregunta final del ejercicio</h2>

¿Cómo podríamos utilizar un modelo de regresión logística para desarrollar una herramienta de predicción continua que alerte sobre clientes potencialmente fugitivos?
Integrar el modelo en el sistema CRM permite monitorear las probabilidades de fuga en tiempo real y configurar alertas automatizadas. Esto facilita la segmentación de clientes y la personalización de estrategias de retención basadas en factores específicos que contribuyen a la probabilidad de fuga.